In [4]:
#! pip install pandas
!pip install tensorflow

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for tensorflow


In [6]:
# Import Libraries
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import (
    BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dropout, Dense
)
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
from tqdm import tqdm
import cv2
import sklearn
import skimage
from skimage.transform import resize
import random
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import seaborn as sns
sns.set()

ModuleNotFoundError: No module named 'pandas'

In [3]:
#Diretório com o dataset utilizado para treinamento
train = "E:\Dados\Training and Validation"

#Diretório com o dataset utilizado para testes
test = "E:\Dados\Testing"


#Define que os dados devem ser carregados a partir de imagens originais
LOAD_FROM_IMAGES = True

# Lendo as imagens de um diretório e convertendo em arrays numéricos (x) 
# junto com as respectivas classes (y).
def get_data(folder):
    x = []
    y = []
    #Percorrendo o diretório 'folder'
    for folderName in os.listdir(folder):
        if not folderName.startswith("."):
            #Com base no nome do diretório,será definida a classe
            if folderName in ["nofire"]:
                label = 0
            elif folderName in ["fire"]:
                label = 1
            else:
                label = 2
            #Percorrendo as imagens dentro de cada classe
            for image_filename in tqdm(os.listdir(folder +"/" +folderName+"/")):
                new_size = (90, 90)
                img_arr = cv2.imread(folder + "/" + folderName + "/" + image_filename)
                if img_arr is not None:
                    #cada nova imagem é carregada e redimensionada para 128x128 pixels
                    img_arr = cv2.resize(img_arr, new_size)
                    # os arrays de imagem e classe são adicionados às listas 
                    x.append(img_arr)
                    y.append(label)
    x = np.asarray(x)
    y = np.asarray(y)
    return x,y

if LOAD_FROM_IMAGES:
    X_train,y_train = get_data(train)
    X_test, y_test = get_data(test)
    
    np.save("xtrain.npy",X_train)
    np.save("ytrain.npy",y_train)
    np.save("xtest.npy",X_test)
    np.save("ytest.npy",y_test)
else:
    X_train = np.load("xtrain.npy")
    y_train = np.load("ytrain.npy")
    X_test = np.load("xtest.npy")
    y_test = np.load("ytest.npy")

from sklearn.model_selection import train_test_split

# dividindo os dados de treinamento em dados de treinamento e validação. 
# Aqui está  divide em uma proporção de 80% para treinamento e 20% para validação.
X_train, X_valid, y_train, y_valid = train_test_split(X_train,y_train,test_size=0.2,shuffle=True)

# Normaliza os valores dos pixels das imagens dividindo-os por 255.0 
# para trazer todos os valores entre 0 e 1
X_train, X_valid, X_test = X_train / 255.0, X_valid / 255.0, X_test / 255.0

100%|███████████████████████████████████████████████████████████████████████████████| 812/812 [00:04<00:00, 174.01it/s]


In [4]:
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.2, horizontal_flip=True)
datagen.fit(X_train)

In [5]:
model = models.Sequential()

# camada de convolução com 32 filtros, cada um com uma janela de convolução de 11x11. 
# Os filtros são aplicados com um passo de 4 pixels horizontalmente e 4 pixels verticalmente.
model.add(layers.Conv2D(32,(11,11),strides=(4, 4), input_shape=(90,90,3), use_bias=False))
# Adiciona uma camada de normalização em lotes após a camada de convolução para normalizar
# os valores de ativação da camada anterior
model.add(BatchNormalization())
# Aplica a função de ativação ReLU (Rectified Linear Unit) para introduzir não-linearidade.
model.add(layers.Activation('relu'))
# Adiciona uma camada de max pooling para reduzir a dimensão espacial dos mapas de 
# características resultantes. Usa uma janela de pooling de 3x3 e 
# um passo de 2 pixels horizontalmente e 2 pixels verticalmente
model.add(layers.MaxPooling2D((3,3), strides=(2,2)))
# Adiciona outra camada de convolução com 32 filtros, cada um com uma janela de 
# convolução de 5x5. A opção padding="same" mantém a saída da camada com a mesma 
# dimensão espacial da entrada.
model.add(layers.Conv2D(32,(5,5),padding="same", use_bias=False))
model.add(BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D((3,3), strides=(2,2)))
model.add(layers.MaxPooling2D((3,3), strides=(2,2)))

# Transforma as saídas 2D das camadas anteriores em um vetor 1D, 
# preparando os dados para as camadas totalmente conectadas (Dense).
model.add(layers.Flatten())

# Adiciona uma camada densamente conectada com 32 neurônios e função de ativação ReLU.
model.add(layers.Dense(128,activation="relu", use_bias=False))
# Adiciona uma camada de dropout com taxa de 50%, o que ajuda a evitar overfitting 
# ao desativar aleatoriamente metade das unidades de neurônios durante o treinamento.
model.add(Dropout(0.5))

model.add(layers.Dense(64,activation="relu", use_bias=False))
model.add(Dropout(0.5))

model.add(layers.Dense(1,activation='sigmoid', use_bias=False))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 20, 20, 32)        11616     
                                                                 
 batch_normalization (BatchN  (None, 20, 20, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 20, 20, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 9, 9, 32)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 32)          25600     
                                                                 
 batch_normalization_1 (Batc  (None, 9, 9, 32)         1

In [7]:
print(model.layers[-1].output_shape)

(None, 1)


In [8]:
model.compile(optimizer = "adam" , loss = "binary_crossentropy", metrics=["accuracy"])

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor="accuracy",
    patience=50,
    restore_best_weights=True)

batch_size=16
epochs=300

#history =  model.fit(X_train,y_train,validation_data=(X_valid,y_valid),batch_size=batch_size, epochs=epochs,verbose=1,callbacks=[early_stopping])
history = model.fit(datagen.flow(X_train, y_train),validation_data=(X_valid, y_valid),batch_size=batch_size,epochs=epochs, verbose=1, callbacks=[early_stopping])

Epoch 1/300
88/88 [==============================] - 28s 156ms/step - loss: 0.4438 - accuracy: 0.8062 - val_loss: 0.4102 - val_accuracy: 0.8407
Epoch 2/300
88/88 [==============================] - 8s 94ms/step - loss: 0.2868 - accuracy: 0.8986 - val_loss: 0.3443 - val_accuracy: 0.8663
Epoch 3/300
88/88 [==============================] - 8s 95ms/step - loss: 0.2719 - accuracy: 0.9083 - val_loss: 0.3156 - val_accuracy: 0.8905
Epoch 4/300
88/88 [==============================] - 12s 132ms/step - loss: 0.2408 - accuracy: 0.9214 - val_loss: 0.4836 - val_accuracy: 0.8435
Epoch 5/300
88/88 [==============================] - 8s 96ms/step - loss: 0.2284 - accuracy: 0.9239 - val_loss: 0.2345 - val_accuracy: 0.9061
Epoch 6/300
88/88 [==============================] - 9s 99ms/step - loss: 0.2200 - accuracy: 0.9303 - val_loss: 0.2212 - val_accuracy: 0.9090
Epoch 7/300
88/88 [==============================] - 24s 276ms/step - loss: 0.2022 - accuracy: 0.9349 - val_loss: 0.2168 - val_accuracy: 0.9118


88/88 [==============================] - 7s 83ms/step - loss: 0.1328 - accuracy: 0.9523 - val_loss: 0.3407 - val_accuracy: 0.8890
Epoch 59/300
88/88 [==============================] - 7s 84ms/step - loss: 0.1080 - accuracy: 0.9637 - val_loss: 0.3907 - val_accuracy: 0.8492
Epoch 60/300
88/88 [==============================] - 7s 81ms/step - loss: 0.1229 - accuracy: 0.9570 - val_loss: 0.1345 - val_accuracy: 0.9545
Epoch 61/300
88/88 [==============================] - 7s 80ms/step - loss: 0.1155 - accuracy: 0.9612 - val_loss: 0.1554 - val_accuracy: 0.9445
Epoch 62/300
88/88 [==============================] - 7s 81ms/step - loss: 0.1194 - accuracy: 0.9580 - val_loss: 0.1583 - val_accuracy: 0.9474
Epoch 63/300
88/88 [==============================] - 8s 86ms/step - loss: 0.1274 - accuracy: 0.9566 - val_loss: 0.1423 - val_accuracy: 0.9502
Epoch 64/300
88/88 [==============================] - 8s 89ms/step - loss: 0.1198 - accuracy: 0.9580 - val_loss: 0.1715 - val_accuracy: 0.9289
Epoch 65/300

88/88 [==============================] - 7s 83ms/step - loss: 0.0884 - accuracy: 0.9701 - val_loss: 0.1848 - val_accuracy: 0.9360
Epoch 116/300
88/88 [==============================] - 7s 78ms/step - loss: 0.0939 - accuracy: 0.9676 - val_loss: 0.1899 - val_accuracy: 0.9431
Epoch 117/300
88/88 [==============================] - 8s 88ms/step - loss: 0.0933 - accuracy: 0.9651 - val_loss: 0.1406 - val_accuracy: 0.9502
Epoch 118/300
88/88 [==============================] - 7s 79ms/step - loss: 0.0932 - accuracy: 0.9666 - val_loss: 0.1570 - val_accuracy: 0.9445
Epoch 119/300
88/88 [==============================] - 7s 78ms/step - loss: 0.0982 - accuracy: 0.9701 - val_loss: 0.1867 - val_accuracy: 0.9403
Epoch 120/300
88/88 [==============================] - 7s 74ms/step - loss: 0.1016 - accuracy: 0.9648 - val_loss: 0.1490 - val_accuracy: 0.9559
Epoch 121/300
88/88 [==============================] - 7s 81ms/step - loss: 0.0940 - accuracy: 0.9669 - val_loss: 0.1615 - val_accuracy: 0.9474
Epoch 

Epoch 172/300
88/88 [==============================] - 7s 84ms/step - loss: 0.0899 - accuracy: 0.9733 - val_loss: 0.2070 - val_accuracy: 0.9346
Epoch 173/300
88/88 [==============================] - 8s 96ms/step - loss: 0.0902 - accuracy: 0.9691 - val_loss: 0.4752 - val_accuracy: 0.8364
Epoch 174/300
88/88 [==============================] - 9s 104ms/step - loss: 0.0730 - accuracy: 0.9726 - val_loss: 0.4137 - val_accuracy: 0.8649
Epoch 175/300
88/88 [==============================] - 8s 88ms/step - loss: 0.0614 - accuracy: 0.9772 - val_loss: 0.1378 - val_accuracy: 0.9587
Epoch 176/300
88/88 [==============================] - 7s 80ms/step - loss: 0.0796 - accuracy: 0.9755 - val_loss: 0.2374 - val_accuracy: 0.9246
Epoch 177/300
88/88 [==============================] - 7s 79ms/step - loss: 0.0772 - accuracy: 0.9694 - val_loss: 0.1936 - val_accuracy: 0.9531
Epoch 178/300
88/88 [==============================] - 7s 75ms/step - loss: 0.0794 - accuracy: 0.9730 - val_loss: 0.1686 - val_accuracy

Epoch 229/300
88/88 [==============================] - 7s 81ms/step - loss: 0.0673 - accuracy: 0.9797 - val_loss: 0.1473 - val_accuracy: 0.9516
Epoch 230/300
88/88 [==============================] - 7s 79ms/step - loss: 0.0635 - accuracy: 0.9772 - val_loss: 0.1525 - val_accuracy: 0.9531
Epoch 231/300
88/88 [==============================] - 7s 77ms/step - loss: 0.0653 - accuracy: 0.9780 - val_loss: 0.4147 - val_accuracy: 0.8777
Epoch 232/300
88/88 [==============================] - 7s 76ms/step - loss: 0.0654 - accuracy: 0.9812 - val_loss: 0.1560 - val_accuracy: 0.9587
Epoch 233/300
88/88 [==============================] - 7s 78ms/step - loss: 0.0703 - accuracy: 0.9744 - val_loss: 0.2575 - val_accuracy: 0.9175
Epoch 234/300
88/88 [==============================] - 7s 78ms/step - loss: 0.0716 - accuracy: 0.9733 - val_loss: 0.1805 - val_accuracy: 0.9531
Epoch 235/300
88/88 [==============================] - 7s 76ms/step - loss: 0.0647 - accuracy: 0.9790 - val_loss: 0.1460 - val_accuracy:

Epoch 286/300
88/88 [==============================] - 7s 78ms/step - loss: 0.0712 - accuracy: 0.9744 - val_loss: 0.1916 - val_accuracy: 0.9459
Epoch 287/300
88/88 [==============================] - 7s 79ms/step - loss: 0.0525 - accuracy: 0.9836 - val_loss: 0.2069 - val_accuracy: 0.9289
Epoch 288/300
88/88 [==============================] - 7s 75ms/step - loss: 0.0639 - accuracy: 0.9804 - val_loss: 0.1706 - val_accuracy: 0.9516
Epoch 289/300
88/88 [==============================] - 7s 75ms/step - loss: 0.0648 - accuracy: 0.9765 - val_loss: 0.3276 - val_accuracy: 0.8890
Epoch 290/300
88/88 [==============================] - 8s 90ms/step - loss: 0.0548 - accuracy: 0.9822 - val_loss: 0.1747 - val_accuracy: 0.9445
Epoch 291/300
88/88 [==============================] - 7s 77ms/step - loss: 0.0473 - accuracy: 0.9822 - val_loss: 0.1687 - val_accuracy: 0.9573


In [9]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test loss:', round(score[0],4))
print('Test accuracy:', round(score[1],4))

95/95 [==============================] - 1s 10ms/step - loss: 0.1178 - accuracy: 0.9668
Test loss: 0.1178
Test accuracy: 0.9668


In [10]:
y_test_pred = model.predict(X_test)

y_pred = (y_test_pred > 0.5)

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

y_pred.shape

y_test.shape

print(classification_report(y_test, y_pred)) 

48/48 [==============================] - 0s 5ms/step
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       812
           1       0.96      0.97      0.96       693

    accuracy                           0.97      1505
   macro avg       0.97      0.97      0.97      1505
weighted avg       0.97      0.97      0.97      1505



In [12]:
model.save('fire_forest_cnn.h5')

In [11]:
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

[[784  28]
 [ 22 671]]
